In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [3]:
import torch
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm.auto import tqdm

In [4]:
 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class DNN(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(DNN, self).__init__()
        self.input_shape = input_shape
        self.num_classes = num_classes
        
        #defining layers
        self.linear1 = nn.Linear(in_features = self.input_shape, out_features = 392)
        self.linear2 = nn.Linear(in_features= 392, out_features = self.num_classes)
        
        #defining activation function
        self.relu = nn.ReLU()
        
    def forward(self, X):
        X = self.linear1(X)
        X = self.relu(X)
        X = self.linear2(X)
        return X
    

#### Hyperparameters

In [6]:
input_shape = 784
num_classes = 10
learning_rate = 0.001
batch_size = 10000
num_epochs = 100

#### Loading Data (Can also load from digit recognizer but have to do some preprocessing)

In [7]:
#Downloading Data Set of MNIST
train_dataset = datasets.MNIST(root="/kaggle/working/", train = True, transform = transforms.ToTensor(), download = True)
test_dataset = datasets.MNIST(root="/kaggle/working/", train = False, transform = transforms.ToTensor(), download = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /kaggle/working/MNIST/raw/train-images-idx3-ubyte.gz to /kaggle/working/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /kaggle/working/MNIST/raw/train-labels-idx1-ubyte.gz to /kaggle/working/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /kaggle/working/MNIST/raw/t10k-images-idx3-ubyte.gz to /kaggle/working/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /kaggle/working/MNIST/raw/t10k-labels-idx1-ubyte.gz to /kaggle/working/MNIST/raw



In [8]:
#Creating Generator a.k.a Dataloader
train_dataloader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

#### Creating and Instance of model

In [9]:
model = DNN(input_shape = input_shape, num_classes = num_classes)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
model = model.to(device)
for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0
    for batch in train_dataloader:
        batch[0] = batch[0].view(batch[0].shape[0], -1).to(device)
        batch[1] = batch[1].to(device)
        
        inference = model.forward(batch[0])
        
        loss = criterion(inference, batch[1])
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        epoch_loss += loss
    if epoch%10 == 0:
        print(f"epoch:- {epoch.__str__().zfill(3)} loss :- {epoch_loss/len(train_dataloader)}")

  0%|          | 0/100 [00:00<?, ?it/s]

epoch:- 000 loss :- 2.018967390060425
epoch:- 010 loss :- 0.2893773913383484
epoch:- 020 loss :- 0.20374926924705505
epoch:- 030 loss :- 0.15015795826911926
epoch:- 040 loss :- 0.11508109420537949
epoch:- 050 loss :- 0.08999183028936386
epoch:- 060 loss :- 0.07169702649116516
epoch:- 070 loss :- 0.05835160240530968
epoch:- 080 loss :- 0.04758518189191818
epoch:- 090 loss :- 0.03912240266799927


In [27]:
def evaluate(model, dataloader):
    model.eval()
    with torch.no_grad():
        num_correct = 0
        total_example = 0
        for batch in dataloader:
            batch[0] = batch[0].view(batch[0].shape[0], -1).to(device)
            batch[1] = batch[1].to(device)

            inference = model.forward(batch[0])
            #output of max is max_value for each example with index location of the max in the last 10 neuron (y_pred) so no need of using softmax activation
            _, y_pred = torch.max(inference, dim = 1)
            num_correct += (y_pred == batch[1]).sum()
            total_example += inference.shape[0]
            
    print(f"Accuracy:- {num_correct/total_example}")
            

In [29]:
# Evaluation on Test Dataset
evaluate(model, test_dataloader)

Accuracy:- 0.9757999777793884


In [31]:
# Evaluation on Train Dataset
evaluate(model, train_dataloader)

Accuracy:- 0.992900013923645
